# YouTube Playlist to Tip Repository

As an avid music producer, I identified a common challenge in the creative process: the disruption of workflow when seeking quick tips from lengthy tutorial videos. To address this, I spearheaded an innovative project to streamline access to valuable production advice. Drawing inspiration from Ableton's One Thing series, which features diverse genre producers sharing creative tips, I developed a script to transcribe and translate video content into accessible text. This script not only transcribes but also intelligently translates content where needed, ensuring universal accessibility.

To elevate the utility of these transcriptions, I integrated them with OpenAI's GPT model for fine-tuning and enhancing the tips, providing succinct, relevant, and context-rich advice. The culmination of this project is a user-friendly app, seamlessly integrated into Ableton's Max for Live. This integration ensures that invaluable tips are readily available within the digital audio workstation (DAW), preserving the artistic flow and enhancing the music creation process. This tool not only demonstrates my technical prowess in programming and AI but also reflects a deep understanding of the needs and challenges within the music production community

In [ ]:
pip install langdetect # recognize multiple languages from a given text string

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install googletrans # Google Translate's AJAX API.

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pytube # Downloads youtube videos

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade requests certifi

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install git+https://github.com/openai/whisper.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/whisper.git to /private/var/folders/rh/gn0gbrrn0rg6r7x003hh5mb00000gn/T/pip-req-build-w08u_lwa
  Running command git clone -q https://github.com/openai/whisper.git /private/var/folders/rh/gn0gbrrn0rg6r7x003hh5mb00000gn/T/pip-req-build-w08u_lwa
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  distutils: /private/var/folders/rh/gn0gbrrn0rg6r7x003hh5mb00000gn/T/pip-build-env-838haz63/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/rh/gn0gbrrn0rg6r7x003hh5mb00000gn/T/pip-build-env-838haz63/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/rh/gn0gbrrn0rg6r7x003hh5mb00000gn/T/pip-build-env-838haz63/normal'
  distutils: /private/var/folders/rh/gn0gbrrn0rg6

In [ ]:
## Import Libraries
import re
import os
import subprocess
import whisper
from pytube import Playlist
from langdetect import detect
from googletrans import Translator
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import gc # Garbage collection.


/Users/abnormalisdope/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# File Path configurations
video_path = '/Users/abnormalisdope/Desktop/Current Coding Projects/2024 Projects/Ableton_Tip_App/tip_videos'  # Where videos will be downloaded to.
audio_path = '/Users/abnormalisdope/Desktop/Current Coding Projects/2024 Projects/Ableton_Tip_App/tip_audio'  # Where converted audio will be stored

In [ ]:
# Downloading Youtube Videos from Ableton's One Thing Playlist
playlist = Playlist('https://www.youtube.com/playlist?list=PLoh4MB-kbBmJCGq34lLYsMQ12b3nqjOae')
DOWNLOAD_DIR = video_path
BATCH_SIZE = 10  # Number of videos to download at once

total_videos = len(playlist.videos)
for i in range(0, total_videos, BATCH_SIZE):
    # Adjust the end index of the slice to ensure it doesn't go beyond the list
    end_index = min(i + BATCH_SIZE, total_videos)
    batch = playlist.videos[i:end_index]
    for video in batch:
        print(f'Downloading: {video.title}')
        video.streams.filter(type='video', progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(DOWNLOAD_DIR)

    gc.collect()  # Invoke garbage collector


In [ ]:
# Convert MP4 videos to MP3

def convert_batch_to_mp3(video_path, audio_path, batch_size=10):
    files = [f for f in os.listdir(video_path) if f.endswith(".mp4")]
    for i in range(0, len(files), batch_size):
        batch = files[i:i+batch_size]
        for file in batch:
            input_file = os.path.join(video_path, file)
            output_file = os.path.join(audio_path, os.path.splitext(file)[0] + '.mp3')
            subprocess.run(['ffmpeg', '-i', input_file, output_file])
        gc.collect()  # Manual garbage collection


convert_batch_to_mp3(video_path, audio_path, batch_size=10)

In [ ]:
# Transcribe MP3s to text

def transcribe_audio_with_whisper(audio_folder):
    model = whisper.load_model("large")
    transcriptions = []
    total_files = len([file for file in os.listdir(audio_folder) if file.endswith(".mp3")])
    processed = 0

    for file in os.listdir(audio_folder):
        if file.endswith(".mp3"):
            audio_path = os.path.join(audio_folder, file)
            result = model.transcribe(audio_path)
            video_title = os.path.splitext(file)[0]
            formatted_transcription = f"Video Title: {video_title}\nTranscription:\n{result['text']}\n\n"

            transcriptions.append(formatted_transcription)

            processed += 1
            print(f"Transcribed {processed}/{total_files} files.")

    return transcriptions

# Usage
formatted_transcriptions = transcribe_audio_with_whisper(audio_path)


/Users/abnormalisdope/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed 1/94 files.


/Users/abnormalisdope/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed 2/94 files.


/Users/abnormalisdope/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Transcribed 92/94 files.


/Users/abnormalisdope/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed 93/94 files.


/Users/abnormalisdope/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed 94/94 files.


In [ ]:
formatted_transcriptions # This is our list with the stored transcriptions. We will need to convert non english transciptions next

["Video Title: One Thing Kenny Segal – Sampling tails\nTranscription:\n Hi, I'm Kenny. One thing that I've found is that sometimes the best thing to sample is the space in between the actual notes. Today I got my keyboard hooked up through some cool reverb and delay pedals to give us a nice tail to mess with. Now instead of using the attack, we're going to go further into the sound and use just the little bit of the tail where all the cool fuzz and texture is. By then taking this little piece of the tail into a simpler, we have a really interesting ethereal sounding pad. This trick can also be applied to drums. Sometimes I'll take a drum break and delete all the actual hits, just leaving the space in between the drums. I can then take those little bits of ambience and add more texture to my own drum sounds. Once it all comes together, these subtle things are really what help the whole groove sing.\n\n",
 "Video Title: One Thing Ben Casey – Re-slicing\nTranscription:\n Hi, I'm Ben. And 

In [ ]:
formatted_transcriptions[4] # Find 1 non english example to test translation

"Video Title: One Thing Splendore – Transformations\nTranscription:\n Ciao, mi chiamo Mattia e una cosa che mi piace fare è trasformare le tracce midi in audio, applicare degli effetti e poi trasformarle in midi così da ottenere nuovi riff. A volte mi trovo con un riff midi che mi piace, che però necessita di variazione e movimento. Quindi per prima cosa prendo questo riff midi, lo faccio entrare in una traccia audio e lo registro. Quando ho il mio sample audio inserisco una serie di effetti, delay, beat repeater o quello che mi ispira al momento, per rendere il suono più stratificato e interessante. Dopo frizzo la traccia audio e applico l'opzione converti melodie in una nuova traccia midi. Ottenuto il nuovo midi mi assicuro che sia nella stessa scala del brano, inserendo il device scale nella tonalità corretta. Il nuovo pattern può essere un ottimo aggiunto a variazione, oppure può fungere da contrapunto alla melodia originale. Se non sono ancora soddisfatto del risultato ottenuto, i

In [ ]:
pip install deep-translator # First language library had issues using deep translator instead


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 42 kB 716 kB/s eta 0:00:01
     |████████████████████████████████| 147 kB 2.4 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Testing translation with 1 example

from deep_translator import GoogleTranslator
from langdetect import detect

def translate_text(text, target_language='en'):
    detected_language = detect(text)
    print(f"Detected language: {detected_language}")

    if detected_language != target_language:
        try:
            translated_text = GoogleTranslator(source=detected_language, target=target_language).translate(text)
            return translated_text
        except Exception as e:
            print(f"Translation failed: {e}")
            return text
    else:
        return text

# Test translation on a specific line
test_line = formatted_transcriptions[4]
title, transcript_text = test_line.split('\n', 1)
translated_text = translate_text(transcript_text)

# Display the results
print("Original:")
print(test_line)
print("\nTranslated:")
print(f"{title}\n{translated_text}")


Detected language: it
Original:
Video Title: One Thing Splendore – Transformations
Transcription:
 Ciao, mi chiamo Mattia e una cosa che mi piace fare è trasformare le tracce midi in audio, applicare degli effetti e poi trasformarle in midi così da ottenere nuovi riff. A volte mi trovo con un riff midi che mi piace, che però necessita di variazione e movimento. Quindi per prima cosa prendo questo riff midi, lo faccio entrare in una traccia audio e lo registro. Quando ho il mio sample audio inserisco una serie di effetti, delay, beat repeater o quello che mi ispira al momento, per rendere il suono più stratificato e interessante. Dopo frizzo la traccia audio e applico l'opzione converti melodie in una nuova traccia midi. Ottenuto il nuovo midi mi assicuro che sia nella stessa scala del brano, inserendo il device scale nella tonalità corretta. Il nuovo pattern può essere un ottimo aggiunto a variazione, oppure può fungere da contrapunto alla melodia originale. Se non sono ancora soddisfa

In [ ]:
# Translating non-english text and create list will all tips in english

def translate_text(text, target_language='en'):
    detected_language = detect(text)
    if detected_language != target_language:
        try:
            translated_text = GoogleTranslator(source=detected_language, target=target_language).translate(text)
            return translated_text
        except Exception as e:
            print(f"Translation failed: {e}")
            return text
    else:
        return text

# List for the updated transcriptions
updated_transcriptions = []

for transcription in formatted_transcriptions:
    title, transcript_text = transcription.split('\n', 1)
    translated_text = translate_text(transcript_text)
    updated_entry = f"{title}\n{translated_text}\n"
    updated_transcriptions.append(updated_entry)

# Now, updated_transcriptions contains all entries with non-English texts translated to English


Translation failed: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=en&sl=es&q=Transcription%3A%0A+Hola%2C+soy+Roman%2C+y+una+cosa+que+me+gusta+hacer+es+crear+polirritmos+en+la+vista+Arrangement+de+Live.+Primero%2C+empiezo+con+algunas+pistas+vac%C3%ADas.+Acorto+la+barra+de+loop+a+medio+bar+y+la+rejilla+a+un+32avo.+Entonces%2C+escojo+algunos+samples+de+percusi%C3%B3n%2C+los+llevo+al+principio+del+track+y+corto+los+samples+largos+a+un+16avo+aproximadamente.+Selecciono+una+secci%C3%B3n+un+poquito+m%C3%A1s+larga+que+el+primer+sample+y+lo+duplico+repetidamente+con+comando+D+hasta+que+el+loop+se+llena.+Hago+lo+mismo+con+el+siguiente+sample%2C+seleccionando+una+largada+distinta%2C+definiendo+el+ritmo+con+la+convivencia+de+largadas.+Para+crear+ritmos+m%C3%A1s+complejos+o+m%C3%A9tricas+distintas%2C+cambio+la+rejilla+a+tresillos.+Finalmente%2C+muevo+los+samples+y+muteo+algunos+hasta+que+estoy+contento+con+el+resultado.+Me+gusta+este+m%C3%A9todo+po

In [ ]:
# Adjusting code for memory and timing out concerns

import time
from deep_translator import GoogleTranslator
from langdetect import detect

def translate_text(text, target_language='en'):
    try:
        detected_language = detect(text)
        if detected_language != target_language:
            translated_text = GoogleTranslator(source=detected_language, target=target_language).translate(text)
            return translated_text
        else:
            return text
    except Exception as e:
        print(f"Translation failed: {e}")
        return text

def process_batch(transcriptions, start, end):
    batch_translations = []
    for transcription in transcriptions[start:end]:
        title, transcript_text = transcription.split('\n', 1)
        translated_text = translate_text(transcript_text)
        batch_translations.append(f"{title}\n{translated_text}\n")
    return batch_translations

# Parameters
batch_size = 10
pause_time = 2  # seconds

# Batch processing
updated_translations = []
num_batches = len(formatted_transcriptions) // batch_size + (1 if len(formatted_transcriptions) % batch_size != 0 else 0)

for i in range(num_batches):
    start_index = i * batch_size
    end_index = start_index + batch_size
    updated_translations.extend(process_batch(formatted_transcriptions, start_index, end_index))
    time.sleep(pause_time)  # Delay between batches

# Check a few entries
for item in updated_translations[:5]:
    print(item)



Video Title: One Thing Kenny Segal – Sampling tails
Transcription:
 Hi, I'm Kenny. One thing that I've found is that sometimes the best thing to sample is the space in between the actual notes. Today I got my keyboard hooked up through some cool reverb and delay pedals to give us a nice tail to mess with. Now instead of using the attack, we're going to go further into the sound and use just the little bit of the tail where all the cool fuzz and texture is. By then taking this little piece of the tail into a simpler, we have a really interesting ethereal sounding pad. This trick can also be applied to drums. Sometimes I'll take a drum break and delete all the actual hits, just leaving the space in between the drums. I can then take those little bits of ambience and add more texture to my own drum sounds. Once it all comes together, these subtle things are really what help the whole groove sing.



Video Title: One Thing Ben Casey – Re-slicing
Transcription:
 Hi, I'm Ben. And one thing I

In [ ]:
updated_translations[:5] # This is our completed translation. Next let's clean things up and put this into a csv

["Video Title: One Thing Kenny Segal – Sampling tails\nTranscription:\n Hi, I'm Kenny. One thing that I've found is that sometimes the best thing to sample is the space in between the actual notes. Today I got my keyboard hooked up through some cool reverb and delay pedals to give us a nice tail to mess with. Now instead of using the attack, we're going to go further into the sound and use just the little bit of the tail where all the cool fuzz and texture is. By then taking this little piece of the tail into a simpler, we have a really interesting ethereal sounding pad. This trick can also be applied to drums. Sometimes I'll take a drum break and delete all the actual hits, just leaving the space in between the drums. I can then take those little bits of ambience and add more texture to my own drum sounds. Once it all comes together, these subtle things are really what help the whole groove sing.\n\n\n",
 "Video Title: One Thing Ben Casey – Re-slicing\nTranscription:\n Hi, I'm Ben. An

In [ ]:
# Cleaning text removing \nTranscription:\n mistake

# Function to format each entry
def format_entry(entry):
    parts = entry.split('\nTranscription:\n')
    return [parts[0].strip(), parts[1].strip()] if len(parts) == 2 else ["", ""]

# Format all entries
formatted_entries = [format_entry(entry) for entry in updated_translations]

In [ ]:
formatted_entries

[['Video Title: One Thing Kenny Segal – Sampling tails',
  "Hi, I'm Kenny. One thing that I've found is that sometimes the best thing to sample is the space in between the actual notes. Today I got my keyboard hooked up through some cool reverb and delay pedals to give us a nice tail to mess with. Now instead of using the attack, we're going to go further into the sound and use just the little bit of the tail where all the cool fuzz and texture is. By then taking this little piece of the tail into a simpler, we have a really interesting ethereal sounding pad. This trick can also be applied to drums. Sometimes I'll take a drum break and delete all the actual hits, just leaving the space in between the drums. I can then take those little bits of ambience and add more texture to my own drum sounds. Once it all comes together, these subtle things are really what help the whole groove sing."],
 ['Video Title: One Thing Ben Casey – Re-slicing',
  "Hi, I'm Ben. And one thing I like to do is c

In [ ]:
# Data will be stored in a CSV file, then analyzed by ChatGPT for grammar and content improvements

import csv

# Specify the destination file path
csv_filename = '/Users/abnormalisdope/Desktop/Current Coding Projects/2024 Projects/Ableton_Tip_App/transcriptions_raw.csv'  # Update this path

# Write to CSV file
with open(csv_filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Video Title', 'Transcription'])  # Header
    for entry in formatted_entries:
        writer.writerow(entry)

print(f"Transcriptions saved to {csv_filename}")


Transcriptions saved to /Users/abnormalisdope/Desktop/Current Coding Projects/2024 Projects/Ableton_Tip_App/transcriptions_raw.csv


# Conclusion
## In conclusion, this Python project has enabled me to transform a collection of Ableton producer tips into a valuable resource. By converting, translating, and posting these tips on a dedicated website, I have created a convenient hub of knowledge for myself and my friends to overcome writer's block and enhance our music production skills. Moving forward, I look forward to expanding this resource with even more tips, ensuring that it continues to serve as a helpful source of advice and inspiration in our creative endeavors.

site: https://dans314.pythonanywhere.com/


